In [18]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

from sklearn.model_selection import train_test_split, GridSearchCV
import datetime

import lightgbm as lgb
from sklearn import preprocessing
from sklearn import metrics


In [2]:
!dir

 ドライブ D のボリューム ラベルがありません。
 ボリューム シリアル番号は 3417-05D0 です

 D:\PythonTraining\Signate\【SOTA】アップル 引越し需要予測 のディレクトリ

2020/10/29  19:30    <DIR>          .
2020/10/29  19:30    <DIR>          ..
2020/10/29  19:30    <DIR>          .ipynb_checkpoints
2020/07/24  19:52            36,738 20200724.ipynb
2020/10/29  19:29            34,818 20201029.ipynb
2020/10/27  20:18            10,921 sample_submit.csv
2020/10/27  20:18             7,351 test.csv
2020/10/27  20:18            48,706 train.csv
               5 個のファイル             138,534 バイト
               3 個のディレクトリ  947,619,794,944 バイトの空き領域


In [45]:
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
submit_df = pd.read_csv('sample_submit.csv', header=None)

In [4]:
train

,datetime,y,client,close,price_am,price_pm
0,2010-07-01,17,0,0,-1,-1
1,2010-07-02,18,0,0,-1,-1
2,2010-07-03,20,0,0,-1,-1
3,2010-07-04,20,0,0,-1,-1
4,2010-07-05,14,0,0,-1,-1
...,...,...,...,...,...,...
2096,2016-03-27,94,1,0,5,4
2097,2016-03-28,86,1,0,4,4
2098,2016-03-29,98,1,0,4,4
2099,2016-03-30,99,1,0,5,4


In [5]:
test

,datetime,client,close,price_am,price_pm
0,2016-04-01,1,0,3,2
1,2016-04-02,0,0,5,5
2,2016-04-03,1,0,2,2
3,2016-04-04,1,0,1,1
4,2016-04-05,0,0,1,1
...,...,...,...,...,...
360,2017-03-27,0,0,5,4
361,2017-03-28,0,0,4,4
362,2017-03-29,0,0,3,3
363,2017-03-30,0,0,4,3


In [6]:
test['y'] = -999

In [7]:
all_df = pd.concat([train, test], axis=0).reset_index(drop=True)
all_df

,datetime,y,client,close,price_am,price_pm
0,2010-07-01,17,0,0,-1,-1
1,2010-07-02,18,0,0,-1,-1
2,2010-07-03,20,0,0,-1,-1
3,2010-07-04,20,0,0,-1,-1
4,2010-07-05,14,0,0,-1,-1
...,...,...,...,...,...,...
2461,2017-03-27,-999,0,0,5,4
2462,2017-03-28,-999,0,0,4,4
2463,2017-03-29,-999,0,0,3,3
2464,2017-03-30,-999,0,0,4,3


In [8]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2466 entries, 0 to 2465
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   datetime  2466 non-null   object
 1   y         2466 non-null   int64 
 2   client    2466 non-null   int64 
 3   close     2466 non-null   int64 
 4   price_am  2466 non-null   int64 
 5   price_pm  2466 non-null   int64 
dtypes: int64(5), object(1)
memory usage: 115.7+ KB


In [9]:
all_df['datetime'] = all_df['datetime'].apply(lambda x: pd.to_datetime(x))

In [10]:
all_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2466 entries, 0 to 2465
Data columns (total 6 columns):
 #   Column    Non-Null Count  Dtype         
---  ------    --------------  -----         
 0   datetime  2466 non-null   datetime64[ns]
 1   y         2466 non-null   int64         
 2   client    2466 non-null   int64         
 3   close     2466 non-null   int64         
 4   price_am  2466 non-null   int64         
 5   price_pm  2466 non-null   int64         
dtypes: datetime64[ns](1), int64(5)
memory usage: 115.7 KB


In [11]:
all_df['week'] = all_df['datetime'].apply(lambda x: x.strftime('%a'))

In [27]:
all_df['year'] = all_df['datetime'].apply(lambda x: x.year)
all_df['month'] = all_df['datetime'].apply(lambda x: x.month)
all_df['day'] = all_df['datetime'].apply(lambda x: x.day)
all_df

,datetime,y,client,close,price_am,price_pm,week,year,month,day
0,2010-07-01,17,0,0,-1,-1,4,2010,7,1
1,2010-07-02,18,0,0,-1,-1,0,2010,7,2
2,2010-07-03,20,0,0,-1,-1,2,2010,7,3
3,2010-07-04,20,0,0,-1,-1,3,2010,7,4
4,2010-07-05,14,0,0,-1,-1,1,2010,7,5
...,...,...,...,...,...,...,...,...,...,...
2461,2017-03-27,-999,0,0,5,4,1,2017,3,27
2462,2017-03-28,-999,0,0,4,4,5,2017,3,28
2463,2017-03-29,-999,0,0,3,3,6,2017,3,29
2464,2017-03-30,-999,0,0,4,3,4,2017,3,30


In [28]:
train = all_df[all_df['y'] != -999]
test = all_df[all_df['y'] == -999]


In [29]:
train

,datetime,y,client,close,price_am,price_pm,week,year,month,day
0,2010-07-01,17,0,0,-1,-1,4,2010,7,1
1,2010-07-02,18,0,0,-1,-1,0,2010,7,2
2,2010-07-03,20,0,0,-1,-1,2,2010,7,3
3,2010-07-04,20,0,0,-1,-1,3,2010,7,4
4,2010-07-05,14,0,0,-1,-1,1,2010,7,5
...,...,...,...,...,...,...,...,...,...,...
2096,2016-03-27,94,1,0,5,4,3,2016,3,27
2097,2016-03-28,86,1,0,4,4,1,2016,3,28
2098,2016-03-29,98,1,0,4,4,5,2016,3,29
2099,2016-03-30,99,1,0,5,4,6,2016,3,30


In [30]:
train.isnull().any()

datetime    False
y           False
client      False
close       False
price_am    False
price_pm    False
week        False
year        False
month       False
day         False
dtype: bool

In [31]:
for col in train.columns:
    print(train[col].value_counts())

2015-10-18    1
2011-03-28    1
2011-10-06    1
2012-11-09    1
2010-07-23    1
             ..
2015-01-27    1
2014-04-22    1
2013-07-25    1
2010-12-20    1
2011-10-29    1
Name: datetime, Length: 2101, dtype: int64
21     59
39     54
29     53
22     52
31     51
       ..
100     1
98      1
90      1
76      1
109     1
Name: y, Length: 99, dtype: int64
0    1895
1     206
Name: client, dtype: int64
0    2074
1      27
Name: close, dtype: int64
 0    945
 1    587
 2    250
-1    211
 3     46
 4     33
 5     29
Name: price_am, dtype: int64
 0    1191
 1     496
-1     212
 2     125
 3      34
 4      30
 5      13
Name: price_pm, dtype: int64
4    301
5    300
3    300
1    300
6    300
2    300
0    300
Name: week, dtype: int64
2012    366
2015    365
2013    365
2011    365
2014    365
2010    184
2016     91
Name: year, dtype: int64
7     186
3     186
1     186
12    186
10    186
8     186
11    180
9     180
2     170
5     155
6     150
4     150
Name: month, dtype: in

In [32]:
# カテゴリカラムの前処理
categorical_features = ['week']
for col in categorical_features:
    lbl = preprocessing.LabelEncoder()
    lbl.fit(all_df[col])
    lbl.transform(all_df[col])
    all_df[col]=lbl.transform(all_df[col])

In [33]:
# 訓練データ、テストデータの分割
train_df = all_df[all_df['y']!=-999]
test_df = all_df[all_df['y']==-999]

In [34]:
y_train = train_df['y']
X_train = train_df.drop(['y', 'datetime'], axis=1)
X_test = test_df.drop(['y', 'datetime'], axis=1)

In [35]:
X_train

,client,close,price_am,price_pm,week,year,month,day
0,0,0,-1,-1,4,2010,7,1
1,0,0,-1,-1,0,2010,7,2
2,0,0,-1,-1,2,2010,7,3
3,0,0,-1,-1,3,2010,7,4
4,0,0,-1,-1,1,2010,7,5
...,...,...,...,...,...,...,...,...
2096,1,0,5,4,3,2016,3,27
2097,1,0,4,4,1,2016,3,28
2098,1,0,4,4,5,2016,3,29
2099,1,0,5,4,6,2016,3,30


In [37]:
y_train

0        17
1        18
2        20
3        20
4        14
       ... 
2096     94
2097     86
2098     98
2099     99
2100    105
Name: y, Length: 2101, dtype: int64

In [38]:
# 訓練データからデータを分割
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.3, random_state=0)

In [39]:
# 使用モデルはLGB（パラメータチューニング無）
lgb_train = lgb.Dataset(X_train, y_train, categorical_feature=categorical_features)
lgb_eval = lgb.Dataset(X_valid, y_valid, reference=lgb_train, categorical_feature=categorical_features)

params = {
    'objective': 'regression',
    'learning_rate': 0.1, # 学習率
    'max_depth': -1, # 木の数 (負の値で無制限)
    'num_leaves': 9, # 枝葉の数
    'metric': ('mean_absolute_error', 'mean_squared_error', 'rmse'),
    #メトリック https://lightgbm.readthedocs.io/en/latest/Parameters.html#metric-parameters
    'drop_rate': 0.15,
    'verbose': 0
}

model = lgb.train(
    params, lgb_train,
    valid_sets=[lgb_train, lgb_eval],
    verbose_eval=10,
    num_boost_round=1000,
    early_stopping_rounds=10
)

y_pred = model.predict(X_test, num_iteration=model.best_iteration)

[LightGBM] [Warning] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000121 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
Training until validation scores don't improve for 10 rounds
[10]	training's l1: 8.22606	training's l2: 114.406	training's rmse: 10.6961	valid_1's l1: 8.46352	valid_1's l2: 118.215	valid_1's rmse: 10.8727
[20]	training's l1: 6.64832	training's l2: 72.8382	training's rmse: 8.53453	valid_1's l1: 6.75511	valid_1's l2: 76.3892	valid_1's rmse: 8.74009
[30]	training's l1: 5.91955	training's l2: 58.7409	training's rmse: 7.66426	valid_1's l1: 6.11861	valid_1's l2: 63.278	valid_1's rmse: 7.95475
[40]	training's l1: 5.53097	training's l2: 51.8209	training's rmse: 7.19868	valid_1's l1: 5.77167	valid_1's l2: 57.6276	valid_1's rmse: 7.59128
[50]	training's l1: 5.31641	training's l2: 47.8328	training's rmse: 6.91612	valid_1's l1: 5.56934	valid_1's l2: 54.5596	valid_1's rmse:

D:\anaconda3\lib\site-packages\lightgbm\basic.py:1551: UserWarning: Using categorical_feature in Dataset.
  warnings.warn('Using categorical_feature in Dataset.')
D:\anaconda3\lib\site-packages\lightgbm\basic.py:1286: UserWarning: Overriding the parameters from Reference Dataset.
  warnings.warn('Overriding the parameters from Reference Dataset.')
D:\anaconda3\lib\site-packages\lightgbm\basic.py:1098: UserWarning: categorical_column in param dict is overridden.
  warnings.warn('{} in param dict is overridden.'.format(cat_alias))


In [40]:
y_pred

array([72.28215495, 73.31575401, 67.00150248, 62.63090487, 60.7990334 ,
       62.694859  , 61.57379834, 61.32422358, 65.87925178, 63.20735329,
       46.23665473, 45.62437799, 45.98913566, 44.86807499, 57.04207846,
       64.75000365, 62.23727538, 44.16876407, 44.78937149, 45.31465528,
       43.05036961, 61.51494853, 66.98577283, 66.20879439, 60.71413432,
       50.04106271, 61.58894206, 66.22731263, 79.83379906, 79.48537711,
       58.16975526, 43.55985163, 43.01092992, 47.84996965, 32.28792355,
       34.52099959, 53.54646032, 42.68257347, 35.71216314, 31.32857373,
       30.7186836 , 31.32503414, 49.05688066, 53.98503168, 51.0013584 ,
       39.06619789, 34.41205748, 34.22874884, 34.83509937, 53.99997976,
       59.08047441, 56.3253749 , 45.72292331, 43.30933589, 43.27723744,
       43.88358798, 56.96705977, 72.31281772, 70.52701303, 65.50439461,
       61.75052754, 50.3260007 , 34.08101083, 45.2896005 , 60.09395141,
       44.01004402, 34.37587609, 29.6877095 , 29.50440086, 30.27

In [46]:
submit_df

,0,1
0,2016-04-01,65.989206
1,2016-04-02,73.076330
2,2016-04-03,62.837857
3,2016-04-04,57.607645
4,2016-04-05,56.316084
...,...,...
360,2017-03-27,95.172858
361,2017-03-28,95.172858
362,2017-03-29,95.172858
363,2017-03-30,95.172858


In [47]:
submit_df[1]=y_pred

In [48]:
submit_df.to_csv('submit_20201029_01.csv',index=False,header=None)